In [2]:
import mtg
import json
import re

In [3]:
data = json.loads(open('cards.json', 'r').read())

In [4]:
cards: list[mtg.Card] = []
for chunk in data:
    if 'card_faces' in chunk.keys():
        continue
    try:
        cards += [mtg.Card.from_json(chunk)]
    except:
        print(json.dumps(chunk, indent=4))

In [33]:
def regex_search(regex: str):
    for card in cards:
        m = re.match(regex, card.text)
        if not m:
            continue
        print(card.text)
        print(m.groups())


In [14]:
print(card_types_r)

(?:(?:\bcreature\b|\bnoncreature\b|\bland\b|\bnonland\b)[,]? )+card


In [54]:
card_types = [
    'creature',
    'land',
    'basic land'
]
card_types_r = '((?:(?:'
for i, t in enumerate(card_types):
    if i != 0:
        card_types_r += '|'
    card_types_r += f'\\b{t}\\b|\\bnon{t}\\b'
card_types_r += r')[,]? )+)card'
# card_types_r = r"((.+)[,]? )+card"

def card_type_matching(line: str, acc: int=0) -> tuple[str, list]:
    m = re.search(card_types_r, line)
    if not m:
        return line, []
    s = m.groups()[0] + 'card'
    ts = m.groups()[0][:-1].split(', ')
    result = []
    key = f'%{acc}%'
    result += [{
        'type': 'card_types',
        'types': ts
    }]
    res_line = line.replace(s, key, 1)
    res_line, n = card_type_matching(res_line, acc+1)
    if n:
        result += n
    return res_line, result

for card in cards[:100]:
    lines = card.text.split('\n')
    for line in lines:
        if line == '': continue
        l, li = card_type_matching(line)
        if not li: continue
        print(line)
        print(l)
        print(json.dumps(li, indent=4))
        print()
# def card_type_matching(line: str) -> tuple[str, list]:
#     m = re.search(card_types_r, line)
#     if not m:
#         return line, [{
#             'type': 'text',
#             'text': line
#         }]
#     s = m.groups()[0] + 'card'
#     ts = m.groups()[0][:-1].split(', ')
#     splits = line.split(s)
#     result = []
#     result += [{
#         'type': 'text',
#         'text': splits[0]
#     }]
#     result += [{
#         'type': 'card_types',
#         'types': ts
#     }]
#     l, n = card_type_matching(splits[1])

#     if n:
#         result += n
#     return l.replace(s, f'[CONDITION{}]') result

# for card in cards[:100]:
#     lines = card.text.split('\n')
#     for line in lines:
#         print(line)
#         print(json.dumps(card_type_matching(line), indent=4))

creature card
{T}: Put target creature card from your graveyard on top of your library. Activate only during your turn, before attackers are declared.
{T}: Put target %0% from your graveyard on top of your library. Activate only during your turn, before attackers are declared.
[
    {
        "type": "card_types",
        "types": [
            "creature"
        ]
    }
]

nonland card
When [CARDNAME] enters the battlefield, it connives. (Draw a card, then discard a card. If you discarded a nonland card, put a +1/+1 counter on this creature.)
When [CARDNAME] enters the battlefield, it connives. (Draw a card, then discard a card. If you discarded a %0%, put a +1/+1 counter on this creature.)
[
    {
        "type": "card_types",
        "types": [
            "nonland"
        ]
    }
]

basic land card
Search your library for a basic land card, put it onto the battlefield tapped, then shuffle.
Search your library for a %0%, put it onto the battlefield tapped, then shuffle.
[
    {
   

In [48]:
'Put target creature card from your graveyard on top of your libra'.replace('creature card', '', 1)

'Put target  from your graveyard on top of your libra'

In [36]:
for card in cards:
    if ' or ' in card.text:
        print(card.text)
        print()

Flying (This creature can't be blocked except by creatures with flying or reach.)

{T}: Target creature loses your choice of flying, first strike, or trample until end of turn.

Draw a card if an opponent has cast a blue or black spell this turn. Spells you control can't be countered this turn. You and permanents you control gain hexproof from blue and from black until end of turn. (You and they can't be the targets of blue or black spells or abilities your opponents control.)

[CARDNAME] can't block and can't be blocked.
You may cast [CARDNAME] from your graveyard as long as you control a black or green permanent.

Choose one or both —
• Create a token that's a copy of target artifact.
• Create a token that's a copy of target creature, except it's an artifact in addition to its other types.

Search your library for a Forest or Plains card, reveal it, put it into your hand, then shuffle.

({B/P} can be paid with either {B} or 2 life.)
{B/P}{B/P}{B/P}{B/P}: Reveal a card you own from ou

In [7]:
whenever_r = r'Whenever ([^.]*)\.'

whenever_common = [
    'attacks',
    'blocks'
]

def whenever_matcher(line: str) -> dict:
    m = re.match(whenever_r, line)
    if not m:
        return {}
    g = m.groups()
    effect = g[0]
    # if effect.count(',') < 2:
    #     return {}
    # print(effect)
    # print(line + '\n')
    card_type_matching(effect)
    
    # if not condition in whenever_common:
    #     return {}
    # print(condition, effect)
    

('creature ',)
{T}: Put target creature card from your graveyard on top of your library. Activate only during your turn, before attackers are declared.

('nonland ',)
When [CARDNAME] enters the battlefield, it connives. (Draw a card, then discard a card. If you discarded a nonland card, put a +1/+1 counter on this creature.)

('land ',)
Search your library for a basic land card, put it onto the battlefield tapped, then shuffle.

('land ',)
When [CARDNAME] enters the battlefield, you may search your library for a basic land card, put that card onto the battlefield tapped, then shuffle.

('creature ',)
When [CARDNAME] enters the battlefield, put target creature card from an opponent's graveyard onto the battlefield under your control. It gains haste. At the beginning of your next end step, exile it.

('land ',)
{4}, {T}: Look at the top card of your library. If it's a land card, you may put it onto the battlefield tapped. If you don't put the card onto the battlefield, you may put it on 